### This notebook illustrates use of slackclient api to post message to private slack workgroups

### including correct formatting for attachments json

**slack documentation:**
slackclient pypi repo:  https://github.com/slackapi/python-slackclient
slackclient python docs:  https://slackapi.github.io/python-slackclient/

### SETUP - slack
create new webhook, get slack api token of the form *`xoxp-####-####-####-####`*

on slack website, configure webhook or bot setup, including permissions and channels
for basic messaging - slack bot must have user:read and user:write permissions
to get channel id for channel you want to write to; 
from slack, right click on channel and `open link`; 
last part of the URL in browser is the channel ID

### SETUP - environment
put confidential slack info in environment variables, not jupyter notebook

create env var named `SLACK_API_TOKEN`; value = token from slack

create env var named `SLACK_CHANNEL_ID`; value = channel ID you want to write to in slack


In [1]:
import os
from slackclient import SlackClient
token = os.environ['SLACK_API_TOKEN']
channel_id = os.environ['SLACK_CHANNEL_ID']

import datetime
import json

In [2]:
# confirm token exists in current system
'SLACK_API_TOKEN' in os.environ

True

In [3]:
# confirm channel ID exists in current system 
'SLACK_CHANNEL_ID' in os.environ

True

In [4]:
# test slack API + token to confirm working authentication
# should return json with 'ok': True as first element
sc = SlackClient(token)
print(sc.api_call("api.test")['ok'])    # just print first element of tuple, to keep confidential info secret

True


In [54]:
# define function to post slack message with api

def slack_message(message, attachments, channel_id):
    sc = SlackClient(token)
    sc.api_call('chat.postMessage', 
                text=message,
                attachments = attachments,
                channel=channel_id, 
                username='surfaceowl',
                icon_emoji=':robot_face:')



In [57]:
# create date and day flags for custom message
date = datetime.datetime.now().strftime("%x")
day_of_week = datetime.datetime.now().strftime("%A")

# define message using f-strings and send
custom_message = f"manual test of slackclient in jupyter notebook sent on { day_of_week }, { date}"
slack_message(custom_message, [], channel_id, )


In [27]:
custom_message = f"*bold text* and _italics text_ test with slackclient in jupyter notebook sent on { day_of_week }, { date}"
slack_message(custom_message, [], channel_id)

In [21]:
sc.api_call("channels.list");

In [22]:
# add emoji - `reactions.add does not work; so just use unicode and slack will translate
# emoji cheat sheet:  https://www.webfx.com/tools/emoji-cheat-sheet/
slack_message(":zap: :ghost: :high_brightness: :rocket: :twisted_rightwards_arrows: :signal_strength: :leo: :bangbang:", [], channel_id)

In [60]:
# test sending message attachments (e.g. with JSON input)

# setup test message input
# "fallback" is a required entry by slack

# define sample test message as dictionary
message_text = f"*test message on { date }* :rocket::rocket::rocket:"

# NOTE:  slackclient library wraps the text below with: { "attachments": ...}
# https://stackoverflow.com/questions/41776070/slack-api-attatchments-from-custom-bot-post-as-plain-text
attachment_json = [{
         "text": "test text",
         "title": "test json attachment",
         "fallback": "Fallback text in slack *[Urgent]*: <https://surfaceowl.com|Test out Slack message attachments>",
         "pretext": "Pretext field fills this spot *[Urgent]*: <https://surfaceowl.com>",
         "color": "#7CD197",
         "fields": 
            {
               "title": "Surface Owl's site",
               "title_link": "https://surfaceowl.com",
               "value": "A very powerful application, companies should by this now",
               "short": "false"
            },
         "mrkdwn_in": [
                "title",
                "pretext"
            ]
      }]


# convert test message to json
json_test_message = json.dumps(attachment_json, separators=(',',':'))
# json_test_message = json.loads(attachment_json)

print(json.dumps(json_test_message))

# send message
slack_message(message_text, attachment_json, channel_id)

"[{\"text\":\"test text\",\"title\":\"test json attachment\",\"fallback\":\"Fallback text in slack **[Urgent]**: <https://surfaceowl.com|Test out Slack message attachments>\",\"pretext\":\"Pretext field fills this spot *[Urgent]*: <https://surfaceowl.com>\",\"color\":\"#7CD197\",\"fields\":{\"title\":\"Surface Owl's site\",\"title_link\":\"https://surfaceowl.com\",\"value\":\"A very powerful application, companies should by this now\",\"short\":\"false\"},\"mrkdwn_in\":[\"title\",\"pretext\"]}]"


In [61]:
message_text = "test message #47"
test_message = [
    {
            "fallback": "Required plain-text summary of the attachment.",
            "color": "#2eb886",
            "pretext": "Optional text that appears above the attachment block",
            "author_name": "Bobby Tables",
            "author_link": "http://flickr.com/bobby/",
            "author_icon": "http://flickr.com/icons/bobby.jpg",
            "title": "Slack API Documentation",
            "title_link": "https://api.slack.com/",
            "text": "Optional text that appears within the attachment",
            "fields": [
                {
                    "title": "Priority",
                    "value": "High",
                    "short": "false"
                }
            ],
            "image_url": "http://my-website.com/path/to/image.jpg",
            "thumb_url": "http://example.com/path/to/thumb.png",
            "footer": "Slack API",
            "footer_icon": "https://platform.slack-edge.com/img/default_application_icon.png",
            "ts": 123456789
}
]

slack_message(message_text, attachment_json, channel_id)

In [13]:
slack_message(json_test_message, channel_id)